# Transformer Experiments

This notebook compares different Transformer architectures for sentiment analysis.

## Objectives
- Train BERT, RoBERTa, XLNet, and ELECTRA models
- Compare performance across models
- Analyze differences in learning curves
- Evaluate on multiple datasets


In [ ]:
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd())))

import torch
import pandas as pd
import numpy as np
from pathlib import Path

from src.data.dataset_loader import load_preprocessed_data
from src.models.transformer_model import BERTForSentiment, RoBERTaForSentiment, XLNetForSentiment, ELECTRAForSentiment
from src.train.train_transformer import train_transformer, evaluate_transformer
from src.train.trainer_utils import create_dataloader
from src.evaluation.metrics import compute_metrics
from src.evaluation.visualization import plot_training_history, plot_confusion_matrix
from src.utils.seed_everything import seed_everything
from src.utils.config_loader import load_config

seed_everything(42)
config = load_config('../config.yaml')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


## 1. Load and Prepare Data


In [ ]:
# Load preprocessed IMDB data (from notebook 02) - already split into train/val/test
print("Loading preprocessed IMDB data (train/val/test splits)...")
train_texts, train_labels = load_preprocessed_data('imdb_train', data_dir='../intermediate/data')
val_texts, val_labels = load_preprocessed_data('imdb_val', data_dir='../intermediate/data')
test_texts, test_labels = load_preprocessed_data('imdb_test', data_dir='../intermediate/data')

print(f"✅ Loaded preprocessed data (already split)")
print(f"Train: {len(train_texts)}, Val: {len(val_texts)}, Test: {len(test_texts)}")


## 2. Train BERT Model


In [ ]:
# Create data loaders
train_loader = create_dataloader(train_texts[:1000], train_labels[:1000], 
                                'bert-base-uncased', batch_size=16, shuffle=True)
val_loader = create_dataloader(val_texts[:200], val_labels[:200],
                               'bert-base-uncased', batch_size=16, shuffle=False)

# Initialize BERT model
model = BERTForSentiment('bert-base-uncased', num_classes=2).to(device)

# Train model
history = train_transformer(model, train_loader, val_loader, device,
                           num_epochs=3, learning_rate=2e-5)


## 3. Visualize Training


## 4. Evaluate on Test Set


In [ ]:
# Evaluate on test set
test_loader = create_dataloader(test_texts[:500], test_labels[:500],
                                'bert-base-uncased', batch_size=16, shuffle=False)

import torch.nn as nn
criterion = nn.CrossEntropyLoss()
test_metrics = evaluate_transformer(model, test_loader, device, criterion)

print("\nTest Results:")
print(f"Test Loss: {test_metrics['loss']:.4f}")
print(f"Test Accuracy: {test_metrics['accuracy']:.4f}")
print(f"Test F1: {test_metrics['f1']:.4f}")


In [ ]:
# Plot training history
plot_training_history(history, save_path='../experiments/plots/bert_training.png')
